In [3]:
%pip install geopy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
from geopy.geocoders import Nominatim

In [5]:
file_path = '215d3c1ea7d0c562774a8c6e4b157ac0.xlsx'
data = pd.ExcelFile(file_path)
df = data.parse('2022')

In [7]:
columns_to_extract = [
   'Miejscowość',
   'Powiat',
   'Województwo'
]

df = df[columns_to_extract]

geolocator = Nominatim(user_agent="poland_cities_locator")

# Get coordinates for each city
city_coordinates = []


for index, entry in df.iterrows():
    # Construct the query string with additional parameters
    query = f"{entry['Miejscowość']}, {entry['Powiat']}, {entry['Województwo']}"
    
    try:
        # Use geocode with country codes to limit the search area
        location = geolocator.geocode(query, addressdetails=True, country_codes='pl', timeout=10)
        
        if location:
            city_coordinates.append({
                "City": entry['Miejscowość'],
                "Region": entry['Powiat'],
                "Latitude": location.latitude,
                "Longitude": location.longitude
            })
        else:
            city_coordinates.append({
                "City": entry['Miejscowość'],
                "Region": entry['Powiat'],
                "Latitude": None,
                "Longitude": None
            })
    except Exception as e:
        print(f"Error geocoding {entry['Miejscowość']}: {e}")
        city_coordinates.append({
            "City": entry['Miejscowość'],
            "Region": entry['Powiat'],
            "Latitude": None,
            "Longitude": None
        })
    
    # Respect Nominatim's usage policy by adding a delay between requests

# Convert the list to a DataFrame
df_cities = pd.DataFrame(city_coordinates)